# EMERFIN_IFS_Search.ipynb

Find the best IFS indicators to cover a list of countries and search terms.

Resources:
* [JSON RESTful Web Service](http://datahelp.imf.org/knowledgebase/articles/667681-using-json-restful-web-service)
* [Data Services News](http://data.imf.org/?sk=A329021F-1ED6-4D6E-B719-5BF5413923B6)

-----------------------------------------------------------------------------------------------------------------

Import packages and obtain metadata dict

In [1]:
import requests     # requests.get('source') method retrieves data from source
import time         # used to pace API requests and avoid disconnection

source = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/DataStructure/IFS'
metafull = (requests.get(source).json()['Structure']['CodeLists']['CodeList'])
items_to_collect = {'areas': 'CL_AREA_IFS', 'units': 'CL_UNIT_MULT'}
name_vars = ('Short Name', 'Full Name', 'Concept', 'Unit', 'Topic')

metadata = {}
for item, code in items_to_collect.items():
    # use item code to filter data_structure result
    item_to_collect = [sub['Code'] for sub in metafull
                       if sub['@id'] == code][0]
    metadata[item] = {
    # Obtain metadata {code: description} from DataStructure method        
        subitem['@value']: subitem['Description']['#text']
        for subitem in item_to_collect
    }

# Collect metadata for indicators separately
meta_indicators = [sub['Code'] for sub in metafull
                   if sub['@id'] == 'CL_INDICATOR_IFS'][0]
metadata['indicators'] = {
    indic['@value']: {
        'Description': indic['Description']['#text']
        } for indic in meta_indicators
}
name_vars = {'Short':2,'Full':3,'Concept':5,'Unit_Name':7,'Topic':8}
for indic in meta_indicators:
    for var, loc in name_vars.items():
        try: 
            metadata['indicators'][indic['@value']][var] = (
            indic['Annotations']['Annotation'][loc]['AnnotationText']['#text'])
        except KeyError:
            continue

Define variables to use in data API request

In [17]:
# Terms to use to filter the descriptions of IFS indicators.
filter_terms = ['Equities', 'Industrial Production', 'Interest Rate', 'Price Ind',
                'Gross Domestic Product', 'Employ', 'Services, Net', 'Rice', 'Soy',
                'Iron', 'Energy', 'Nickel']
filtered_inds = [code for code, descr in metadata['indicators'].items()
                 if any([f_term in descr['Description'] for f_term in filter_terms])]

# ISO2 codes--IFS also contains regional aggregates with IMF specific codes
country_list = ['BR', 'RU', 'ZA', 'MX', 'TR']
cty_list = '+'.join(country_list)  # formatted for API requests

date_lim = '?startPeriod=2014'

# Location of actual IFS data (rather than metadata)
data_src = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/CompactData/IFS/'

In [18]:
filtered_inds

[u'FID_PA',
 u'LCM_IX',
 u'FIGB_PA',
 u'PZPINRG_IX',
 u'AIP_PC_CP_A_PT',
 u'FIMM_FX_PA',
 u'FPEMF_IX',
 u'LEMI_SA_IX',
 u'NGDP_R_SA_IX',
 u'AIPMI_IX',
 u'PZPINGAS_USD_MBTU_RATE',
 u'FILIBOR_6M_PA',
 u'FPOLM_PA',
 u'PZPINGAS_TYPE3_USD_MBTU_RATE',
 u'PZPIOIL_Type3_USD_BBL_RATE',
 u'GBECE_G01_CA_USD',
 u'NGDP_R_SA_AR_XDC',
 u'PZPINICK_USD_TN_RATE',
 u'NGDP_D_SA_IX',
 u'FILIBOR_ON_PA',
 u'FIMM_PA',
 u'PZPICL_USD_TN_RATE',
 u'PPPIHIG_IX',
 u'LOCNSH_EOP_IX',
 u'NGDP_D_IX',
 u'PZPIOIL_TYPE1_USD_BBL_RATE',
 u'NGDP_R_AR_XDC',
 u'GCECE_G01_CA_XDC',
 u'NGDP_R_XDC',
 u'AIMIGOLD_SA_IX',
 u'FITB_PA',
 u'AIPMA_IX',
 u'PZPIRIN_IX',
 u'FPEI_IX',
 u'FILR_FX_PA',
 u'PZPISOIL_IX',
 u'FPESP_IX',
 u'GGECE_G01_AC_XDC',
 u'PZPISMEA_IX',
 u'FPE_IX',
 u'PPPI_IX',
 u'FPEPFTSE100_IX',
 u'PZPIFE_USD_TN_RATE',
 u'FPEPNASC_IX',
 u'NGDP_SA_AR_XDC',
 u'NGDP_R_SA_XDC',
 u'TXGSOYB_R_FOB_IX',
 u'NGDP_R_IX',
 u'AIPMA_SA_IX',
 u'NGDP_AR_XDC',
 u'GCECE_G01_AC_EUR',
 u'GBECE_G01_AC_EUR',
 u'PZPISMEA_USD_TN_RATE',
 u'FIDR_FX_

Make request to data API and save nonblank series to list

In [8]:
# One indicator at a time, full list with valid results from throwing all
# combinations of countries and filtered indicators at the API
valid_series = []
for filt_ind in filtered_inds:
    data_url = '{}.{}.{}.{}'.format(data_src, cty_list, filt_ind, date_lim)
    data = requests.get(data_url).json()['CompactData']['DataSet']
    if 'Series' in data.keys():         # Check if data contain series
        for series in data['Series']:
            if 'Obs' in series.keys():  # Check if series contain observations
                valid_series.append(series)
    time.sleep(0.5)   # Delay requests slightly so server does not block them

Use pandas to build table of best indicators

In [9]:
import pandas as pd  # Pandas to organize results and export to csv

df = pd.DataFrame(valid_series).iloc[:,0:5]
valid_series = None  # clear the extra data from memory
# We prefer more frequent data, and give it a higher score in the sort
df['Freq'] = df['@FREQ'].replace({'M':2, 'Q':1, 'A':0})
df = df.sort_values(['@INDICATOR', '@REF_AREA', 'Freq']).drop_duplicates(
    ['@INDICATOR', '@REF_AREA'], keep='last').reset_index(drop=True)
df['Country'] = df['@REF_AREA'].map(metadata['areas'])
df['Units'] = df[df['@UNIT_MULT']!='0']['@UNIT_MULT'].map(metadata['units'])
# Merge in metadata from indicator level 
meta_df = pd.DataFrame(metadata['indicators']).T
df = pd.concat([df.set_index('@INDICATOR'), meta_df], axis=1, join='inner')
# Generate more clear/specific measures for each series
df['Category'] = df['Full'].str.split(',', expand=True, n=1)[0]
df['Y_Label'] = df['Unit_Name'].str.cat(df['Units'], sep=', ').fillna(df['Unit_Name'])
df['Combined'] = df['@FREQ'].str.cat([df['@REF_AREA'], df.index], sep='.')
df['Chart_Title'] = df['Description'].str.rsplit(',', expand=True, n=1)[0]

In [10]:
Group_dict = {
    'National Accounts':'1-National Accounts',
    'Labor Markets':'2-Labor Markets',
    'Social Indicators':'2-Labor Markets',
    'Economic Activity':'3-Economic Activity',
    'Producer Price Index':'4-Prices',
    'Consumer Prices':'4-Prices',
    'Prices':'4-Prices',
    'Balance of Payments':'5-Balance of Payments',
    'Exchange Rates':'6-Exchange Rates',
    'Financial':'7-Interest Rates',
    'Financial Market Prices': '8-Equities'
}
df['Group'] = df['Category'].map(Group_dict)
df.index.name = '@INDICATOR'
df = df.reset_index().set_index('Combined')

In [13]:
df

,@FREQ,@REF_AREA,@TIME_FORMAT,@UNIT_MULT,Freq,Country,Units,Concept,Description,Full,Short,Topic,Unit_Name,Category,Y_Label,Combined,Chart_Title,Group
AIMIGOLD_SA_IX,M,ZA,P1M,0,2,South Africa,NaN,"Gold Production, Seasonally adjusted","Industrial Production, Mining, Gold Production...","Economic Activity, Industrial Production, Mini...","Gold Production, Seasonally adjusted, Index",Real Sector,Index,Economic Activity,Index,M.ZA.AIMIGOLD_SA_IX,"Industrial Production, Mining, Gold Production...",3-Economic Activity
AIPMA_IX,M,MX,P1M,0,2,Mexico,NaN,Manufacturing,"Industrial Production, Manufacturing, Index","Economic Activity, Industrial Production, Manu...","Manufacturing, Index",Real Sector,Index,Economic Activity,Index,M.MX.AIPMA_IX,"Industrial Production, Manufacturing",3-Economic Activity
AIPMA_SA_IX,M,ZA,P1M,0,2,South Africa,NaN,"Industrial Production, Manufacturing, Seasonal...","Industrial Production, Manufacturing, Seasonal...","Economic Activity, Industrial Production, Manu...","Industrial Production, Manufacturing, Seasonal...",Real Sector,Index,Economic Activity,Index,M.ZA.AIPMA_SA_IX,"Industrial Production, Manufacturing, Seasonal...",3-Economic Activity
AIPMI_IX,M,MX,P1M,0,2,Mexico,NaN,"Industrial Production, Mining","Industrial Production, Mining, Index","Economic Activity, Industrial Production, Mini...","Industrial Production, Mining, Index",Real Sector,Index,Economic Activity,Index,M.MX.AIPMI_IX,"Industrial Production, Mining",3-Economic Activity
AIPMI_SA_IX,M,ZA,P1M,0,2,South Africa,NaN,"Mining, Seasonally adjusted","Industrial Production, Mining, Seasonally adju...","Economic Activity, Industrial Production, Mini...","Mining, Seasonally adjusted, Index",Real Sector,Index,Economic Activity,Index,M.ZA.AIPMI_SA_IX,"Industrial Production, Mining, Seasonally adju...",3-Economic Activity
AIPMMOC_IX,M,MX,P1M,0,2,Mexico,NaN,"Industrial Production, Manufacturing, Non-dura...","Industrial Production, Manufacturing, Non-dura...","Economic Activity, Industrial Production, Manu...","Industrial Production, Manufacturing, Non-dura...",Real Sector,Index,Economic Activity,Index,M.MX.AIPMMOC_IX,"Industrial Production, Manufacturing, Non-dura...",3-Economic Activity
AIP_IX,M,MX,P1M,0,2,Mexico,NaN,Industrial Production,"Industrial Production, Index","Economic Activity, Industrial Production, Index","Industrial Production, Index",Real Sector,Index,Economic Activity,Index,M.MX.AIP_IX,Industrial Production,3-Economic Activity
AIP_IX,M,RU,P1M,0,2,Russian Federation,NaN,Industrial Production,"Industrial Production, Index","Economic Activity, Industrial Production, Index","Industrial Production, Index",Real Sector,Index,Economic Activity,Index,M.RU.AIP_IX,Industrial Production,3-Economic Activity
AIP_IX,M,TR,P1M,0,2,Turkey,NaN,Industrial Production,"Industrial Production, Index","Economic Activity, Industrial Production, Index","Industrial Production, Index",Real Sector,Index,Economic Activity,Index,M.TR.AIP_IX,Industrial Production,3-Economic Activity
AIP_SA_IX,M,BR,P1M,0,2,Brazil,NaN,"Industrial Production, Seasonally adjusted","Industrial Production, Seasonally adjusted, Index","Economic Activity, Industrial Production, Seas...","Industrial Production, Seasonally adjusted, Index",Real Sector,Index,Economic Activity,Index,M.BR.AIP_SA_IX,"Industrial Production, Seasonally adjusted",3-Economic Activity
